# Model Training:

## Introduction:
This notebook showcases the training process of the model. Due to the amount of the original images exceeding $700,000$, this demo will only go over the training process on the artificially made images. Additionally, we'll only be using the $I5$ image datasets as the other datasets are require lot's of computational resources. For this example, we'll be using the artificially generated base images. Once again, to go more into depth, please view the `train_model.py` module. 

## Import Packages:
Please import the packages below.

In [1]:
## Import all necessary packages.
import tensorflow as tf
import os
import sys

sys.path.append(os.path.abspath(os.path.join('..')))
import src.models.cnn_model as cm
import src.utils.file_operations as fo

## Initialize Working Directories:
Please use the following to code to establish the base directory.

In [2]:
## Initialize the base directory to be in .\stock_code.
base_dir = os.path.abspath(os.path.join('.', '..'))

## Uncomment code below to view your current working directory to check if it's correct.
# print(base_dir)

## Initialize and Compile Model:
We'll first initialize our model. Since we're training $I5R5$ images, we'll have to use the model designed for $5$ day interval. This can be done by fetching the $5$ day model from the `StockCNNModel` class in `cnn_model.py` module. After obtaining the required model, we can compile and view it's architecture (shown below).

In [3]:
## Choose and compile the necessary model you want to train.
model = cm.StockCNNModel('five').generate_model()

## Showcase the model's architecture.
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 13, 64)        1024      
                                                                 
 batch_normalization (BatchN  (None, 28, 13, 64)       256       
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 28, 13, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 13, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 13, 128)       123008    
                                                                 
 batch_normalization_1 (Batc  (None, 14, 13, 128)      5

## Preprocessing Training Images:
After loading in our model, we'll have to load in our training images. This can be done using Tensorflow's built-in function `tf.keras.utils.image_dataset_from_directory`. To properly use the function, we'll first need to define the directory that stores our training images and the training image shape (i.e., height and width)

In [4]:
## Define the directory that stores the training images.
train_img_dir = os.path.join(base_dir, 'data', 'processed_img', 'artificial_img', 'I5R5', 'base_img', 'training_dataset')

## Establish the input shape for the model.
input_shape = (32, 15)

Using the parameters defined above, we can now create our training and validation datasets using Tensorflow's built-in functions. Validation will be comprised of $30\%$ of the initial data and our batch size will be $128$.

In [5]:
## Create training and validation dataset using Tensorflow.
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
    train_img_dir, 
    labels = 'inferred', 
    label_mode = 'binary', 
    color_mode = 'grayscale', 
    batch_size = 128, 
    image_size = input_shape, 
    shuffle = True, 
    validation_split = 0.3, 
    subset = 'both', 
    seed = 42
)

Found 585 files belonging to 2 classes.
Using 410 files for training.
Using 175 files for validation.


Now that we have our training and validation datasets, we can begin training the model.

## Training the Model:
To train the model, we'll use the Adam Optimizer with an initial learning rate of $1 \times 10^{-5}$ and binary cross entropy to calculate error. Our metrics will only consist of tracking the model's accuracy. We'll use $100$ epochs for this training process, yet this number can be different in the `train_model` script. 

Before training, we'll also need to establish early stopping. This would make sure to stop the training process if our valdiation loss doesn't improve after two epochs and recover the weights with the best performance. This is shown below.

In [6]:
## Define the early stopping callback.
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss', 
    patience = 2, 
    restore_best_weights = True
)

Using these newly defined variables, we can now begin the training process.

In [7]:
## Train the model.
model.fit(train_ds, epochs = 50, callbacks = [early_stopping], validation_data = val_ds)

Epoch 1/50
4/4 [==============================] - 3s 259ms/step - loss: 0.5383 - accuracy: 0.7268 - val_loss: 0.3762 - val_accuracy: 0.8971
Epoch 2/50
4/4 [==============================] - 0s 125ms/step - loss: 0.1188 - accuracy: 0.9659 - val_loss: 0.1482 - val_accuracy: 0.9714
Epoch 3/50
4/4 [==============================] - 0s 125ms/step - loss: 0.0670 - accuracy: 0.9756 - val_loss: 0.1417 - val_accuracy: 0.9714
Epoch 4/50
4/4 [==============================] - 0s 122ms/step - loss: 0.0618 - accuracy: 0.9756 - val_loss: 0.1008 - val_accuracy: 0.9771
Epoch 5/50
4/4 [==============================] - 0s 32ms/step - loss: 0.0381 - accuracy: 0.9854 - val_loss: 0.1257 - val_accuracy: 0.9771
Epoch 6/50
4/4 [==============================] - 0s 55ms/step - loss: 0.0366 - accuracy: 0.9878 - val_loss: 0.1395 - val_accuracy: 0.9771


## Saving Trained Model:
After training, you can save the model's weights. Please use the function `save_model` from the `file_operations` module to save the existing weights as a `h5` file. This trained model can later be loaded in and be applied to the testing images (see more in `Model_Evaluation` notebook).

In [8]:
## Define the directory that stores the model's weight.
models_out_dir = "[your-path-to-models]\\[test_model_name].h5"

## Use function to save the model's weight to designated directory.
fo.save_model(model, models_out_dir)